## Insert 'Todos' Columns

In [ ]:
def check_and_create_all_municipios_id(session : Session):
    # Verificar se o município com ID 1 existe e, se não, inseri-lo
    municipio = session.query(Municipio).filter_by(id=1).first()
    if not municipio:
        municipio = Municipio(id=1, nome='Todos', UF='MG')
        session.add(municipio)
        session.commit()

def municipios_sum_query_df(session: Session):
    query = (
        session.query(
            Filtro.etapa_de_ensino,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa,
            func.sum(Matricula.quantidade).label('quantidade')
        )
        .join(Matricula, Filtro.id == Matricula.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.etapa_de_ensino,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Filtro.etapa_de_ensino,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
    )

    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results, columns=['etapa_de_ensino', 'ano', 'cor_raca', 'dependencia_administrativa', 'quantidade'])
    df['municipio_id'] = 1
    
    display(df)
    return df

def etapa_de_ensino_sum(session: Session):
    query = (
        session.query(
            Filtro.municipio_id,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa,
            func.sum(Matricula.quantidade).label('quantidade')
        )
        .join(Matricula, Filtro.id == Matricula.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.municipio_id,
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Filtro.municipio_id,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
    )
    
    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results, columns=['municipio_id', 'ano', 'cor_raca', 'dependencia_administrativa', 'quantidade'])
    df['etapa_de_ensino'] = 'Todos'
    
    display(df)
    return df

def municipios_etapa_de_ensino_sum(session: Session):
    query = (
        session.query(
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa,
            func.sum(Matricula.quantidade).label('quantidade')
        )
        .join(Matricula, Filtro.id == Matricula.id_filtro)
        .filter(Filtro.municipio_id != 1)
        .filter(Filtro.etapa_de_ensino != 'Todos')
        .group_by(
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
        .order_by(
            Filtro.ano,
            Matricula.cor_raca,
            Matricula.dependencia_administrativa
        )
    )

    # Executar a query
    results = query.all()
    # Converter resultados em DataFrame
    df = pd.DataFrame(results, columns=['ano', 'cor_raca', 'dependencia_administrativa', 'quantidade'])
    df['etapa_de_ensino'] = 'Todos'
    df['municipio_id'] = 1
    
    display(df)
    return df

def send_combination(session: Session):
    check_and_create_all_municipios_id(session=session)
    query_df = municipios_etapa_de_ensino_sum(session=session)
    bulk_insert_data(full_table_data=query_df)

def send_etapa_de_ensino_sum_to_db(session: Session):
    query_df = etapa_de_ensino_sum(session=session)
    bulk_insert_data(full_table_data=query_df)

def send_municipios_sum_to_db(session: Session):
    check_and_create_all_municipios_id(session=session)
    query_df = municipios_sum_query_df(session=session)
    bulk_insert_data(full_table_data=query_df)

def insert_total():
    with Session(engine) as session:
        send_municipios_sum_to_db(session=session)
        send_etapa_de_ensino_sum_to_db(session=session)
        send_combination(session=session)


In [ ]:
insert_total()